In [1]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from tqdm.notebook import tqdm

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [11]:
np.random.seed(42)
x = np.array([[0.1, 1.2], [1.2, 0.2]])
y = np.array([[0.2], [1.2]])
theta = np.random.uniform(0, 2*np.pi, (2,1))

In [12]:
def anzats(x, y, theta, backend, shots, verbose=False):
    features = qk.QuantumRegister(2, name="features")
    ancilla = qk.QuantumRegister(2, name="ancilla")
    targets = qk.QuantumRegister(1, name="targets")
    swap = qk.QuantumRegister(1, name="swap")
    classical = qk.ClassicalRegister(1)
    
    registers = [features, ancilla, targets, swap, classical]
    circuit = qk.QuantumCircuit(*registers)

    circuit.h(ancilla)
    circuit.h(swap)

    circuit.cry(x[0,0], ancilla[0], features[0])
    circuit.cry(x[0,1], ancilla[0], features[1])

    circuit.x(ancilla[0])

    circuit.cry(x[1,0], ancilla[0], features[0])
    circuit.cry(x[1,1], ancilla[0], features[1])

    circuit.x(ancilla[0])

    circuit.cry(y[0,0], ancilla[1], targets[0])
    circuit.x(ancilla[1])
    circuit.cry(y[1,0], ancilla[1], targets[0])
    circuit.x(ancilla[1])

    circuit.barrier()

  #  circuit.cx(features[0], features[1])
    circuit.ry(theta[0,0], features[0])
    circuit.ry(theta[1,0], features[1])
    circuit.cx(features[0], features[1])

    circuit.barrier()

    circuit.cswap(swap, features[1], targets[0])
    circuit.cswap(swap, ancilla[0], ancilla[1])
    circuit.h(swap)
    circuit.measure(swap, classical)
    
    if verbose:
        print(circuit)

    job = qk.execute(circuit, backend, shots=shots)
    counts = job.result().get_counts(circuit)
    if "0" in counts:
        value = counts["0"] / shots
    else:
        value = 0
    
    return value

In [13]:
backend = Aer.get_backend('qasm_simulator')
shots = 10000
for i in range(100):
    theta[0] += np.pi/2
    a = anzats(x, y, theta, backend, shots)
    theta[0] += -np.pi
    b = anzats(x, y, theta, backend, shots)
    theta[0] += np.pi/2
    dtheta1 = 1/(2*np.sqrt(2))*(a-b)

    theta[1] += np.pi/2
    c = anzats(x, y, theta, backend, shots)
    theta[1] += -np.pi
    d = anzats(x, y, theta, backend, shots)
    theta[1] += np.pi/2
    dtheta2 = 1/(2*np.sqrt(2))*(c-d)
    
    dtheta = np.array([[dtheta1], [dtheta2]])
    theta += 0.5*dtheta
    
    loss = anzats(x, y, theta, backend, shots)
    
    print(loss)  

0.6457
0.657
0.6813
0.6935
0.7028
0.7276
0.7538
0.7663
0.78
0.7965
0.8189
0.8348
0.8401
0.8567
0.8735
0.8832
0.8924
0.904
0.9147
0.922
0.925
0.9316
0.9375
0.9417
0.9491
0.951
0.9514
0.9542
0.9562
0.9595
0.9606
0.9592
0.9646
0.9651
0.9668
0.9663
0.9681
0.9665
0.968
0.9662
0.9654
0.9676
0.9709
0.9748
0.967
0.971
0.9665
0.9694
0.9716
0.9676
0.9719
0.9708
0.965
0.9689
0.9681
0.9687
0.9733
0.9685
0.9733
0.9723
0.9728
0.9706
0.9714
0.9675
0.9664
0.9693
0.9752
0.9706
0.9714
0.9698
0.9726
0.9713
0.9699
0.9697
0.9722
0.9699
0.9728
0.9699
0.9678
0.9711
0.9708
0.9693
0.9678
0.9691
0.9723
0.9713
0.9717
0.9701
0.9716
0.9716
0.9722
0.9704
0.9713
0.9708
0.9704
0.9678
0.9711
0.9694
0.9713
0.9705
